## Embedding Models
For now, utilize gensim as the wrapper library for embedding models.

In [20]:

import gensim.downloader as api
from datetime import datetime

print(datetime.now())
model_fasttext = api.load("fasttext-wiki-news-subwords-300")
print(datetime.now())
model_gigaword = api.load("glove-wiki-gigaword-100")
print(datetime.now())
# model.most_similar(positive=["russia", "river"])


2019-12-16 22:29:34.453684
2019-12-16 22:34:28.428846
2019-12-16 22:35:08.827176


In [12]:
sentence_obama = 'Obama speaks to the media'.lower().split()
sentence_president = 'The president greets the press in Chicago'.lower().split()

model = model_gigaword
# similarity = word_vectors.wmdistance(sentence_obama, sentence_president)
# print("{:.4f}".format(similarity))
distance = model.distance("media", "media")
print("{:.1f}".format(distance))
sim = model.n_similarity(sentence_obama, sentence_president)
print("{:.4f}".format(sim))
vector = model['computer']  # numpy vector of a word
print(vector.shape)
# vector = model2.wv.word_vec('office', use_norm=True)
# print(vector.shape)


0.0
0.8993
(100,)


In [32]:
import pandas as pd
import parse_csm
import parse_imdb
from os import path
from tqdm import tqdm, tqdm_notebook
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
#https://www.wordnik.com/words/sustainability
definitions_word = {
    'sustainability': """Sustainability is the ability to exist constantly. In the 21st century, it refers generally to the capacity for the biosphere and human civilization to coexist. It is also defined as the process of people maintaining change in a homeostasis balanced environment, in which the exploitation of resources, the direction of investments, the orientation of technological development and institutional change are all in harmony and enhance both current and future potential to meet human needs and aspirations.  For many in the field, sustainability is defined through the following interconnected domains or pillars: environment, economic and social, which according to Fritjof Capra  is based on the principles of Systems Thinking. Sub-domains of sustainable development have been considered also: cultural, technological and political. According to Our Common Future, Sustainable development is defined as development that "meets the needs of the present without compromising the ability of future generations to meet their own needs."  Sustainable development may be the organizing principle of sustainability, yet others may view the two terms as paradoxical (i.e., development is inherently unsustainable).""",
    # https://en.wikipedia.org/wiki/Civic_engagement
    'civic engagement': """Civic engagement or civic participation is any individual or group activity addressing issues of public concern. Citizens acting alone or together to protect public values or make a change or difference in the community are common types of civic engagement. Civic engagement includes communities working together in both political and non-political actions. The goal of civic engagement is to address public concerns and promote the quality of the community. """,
    # https://ok2bme.ca/resources/kids-teens/what-does-lgbtq-mean/
    "LGBTQ+": """LGBTQ is the more commonly used term in the community; possibly because it is more user friendly! You may also hear the terms “Queer Community” or “Rainbow Community” used to describe LGBTQ2+ people. This initialism and the various terms are always evolving so don’t try to memorize the list. The most important thing is to be respectful and use the terms that people prefer.""", 
    # https://en.wikipedia.org/wiki/Substance_abuse
    "substance abuse": """Substance abuse, also known as drug abuse, is use of a drug in amounts or by methods which are harmful to the individual or others. It is a form of substance-related disorder. Differing definitions of drug abuse are used in public health, medical and criminal justice contexts. In some cases criminal or anti-social behaviour occurs when the person is under the influence of a drug, and long term personality changes in individuals may occur as well. In addition to possible physical, social, and psychological harm, use of some drugs may also lead to criminal penalties, although these vary widely depending on the local jurisdiction.""", 
    # https://en.wikipedia.org/wiki/Homelessness
    "homelessness": """Homelessness is defined as living in housing that is below the minimum standard or lacks secure tenure. People can be categorized as homeless if they are: living on the streets (primary homelessness); moving between temporary shelters, including houses of friends, family and emergency accommodation (secondary homelessness); living in private boarding houses without a private bathroom and/or security of tenure (tertiary homelessness). The legal definition of homeless varies from country to country, or among different jurisdictions in the same country or region.  According to the UK homelessness charity Crisis, a home is not just a physical space: it also provides roots, identity, security, a sense of belonging and a place of emotional wellbeing.  United States government homeless enumeration studies also include people who sleep in a public or private place not designed for use as a regular sleeping accommodation for human beings. People who are homeless are most often unable to acquire and maintain regular, safe, secure and adequate housing due to a lack of, or an unsteady income. Homelessness and poverty are interrelated. There is no methodological consent on counting the homeless and identifying their special needs, therefore in most of the cities only estimated homeless population is known.""",
    # https://en.m.wikipedia.org/wiki/Bullying
    "bullying": """Bullying is the use of force, coercion, or threat, to abuse, aggressively dominate or intimidate. The behavior is often repeated and habitual. One essential prerequisite is the perception (by the bully or by others) of an imbalance of physical or social power. This imbalance distinguishes bullying from conflict.[1] Bullying is a subcategory of aggressive behavior characterized by the following three minimum criteria: (1) hostile intent, (2) imbalance of power, and (3) repetition over a period of time.[2] Bullying is the activity of repeated, aggressive behavior intended to hurt another individual, physically, mentally, or emotionally. """
}

path_root = path.join('/Users/quinone/Downloads/portia_projects/hackathon/scrape-hand', 'movies')
path_data = path.join(path_root, '..', 'data.191216.1918.json.gz')
df_csm = pd.read_json(path_data)

# path_root = path.join('/Users/quinone/Downloads/portia_projects/hackathon/scrape-hand', 'imdb')
# path_data = path.join(path_root, '..', 'imdb.json.gz')
# df_imdb = pd.read_json(path_data)


In [43]:
def tokenize_models(text, models):
    tokens = list(tokenize(text, lowercase=True))
    for obj in models:
        tokens = [v for v in tokens if v in obj]
    return tokens    

col_keys = list(definitions_word.keys())
print(df_csm.columns)
from gensim.utils import tokenize
list_split = [tokenize_models(definitions_word[col_keys[idx]],
                              [model_fasttext, model_gigaword]) for idx in range(len(col_keys))]
for idx in tqdm_notebook(range(len(df_csm))):
    str_split = tokenize_models(df_csm.iloc[idx]["summary"], [model_fasttext, model_gigaword])
    for def_idx in range(len(col_keys)):
        if idx == 0:
            df_csm[f"{col_keys[def_idx]}-fast"] = 0
            df_csm[f"{col_keys[def_idx]}-giga"] = 0
        df_csm[f"{col_keys[def_idx]}-fast"].iloc[idx] = model_fasttext.n_similarity(str_split, list_split[def_idx])
        df_csm[f"{col_keys[def_idx]}-giga"].iloc[idx] = model_gigaword.n_similarity(str_split, list_split[def_idx])

path_data = path.join(path_root, '..', 'topics.json.gz')
df_csm.to_json(path_data, orient='records')
df_csm.sample(4)


Index(['title', 'brief', 'url', 'summary', 'rating', 'url_imdb', 'keywords',
       'poster', 'age_adult', 'age_adult_count', 'age_child',
       'age_child_count', 'brief_oneline', 'release_year', 'duration',
       'summary_parents', 'scores', 'scores_text', 'questions', 'details',
       'sustainability-fast', 'sustainability-giga', 'civic engagement-fast',
       'civic engagement-giga', 'LGBTQ+-fast', 'LGBTQ+-giga',
       'substance abuse-fast', 'substance abuse-giga', 'homelessness-fast',
       'homelessness-giga', 'bullying-fast', 'bullying-giga'],
      dtype='object')


,title,brief,url,summary,rating,url_imdb,keywords,poster,age_adult,age_adult_count,...,civic engagement-fast,civic engagement-giga,LGBTQ+-fast,LGBTQ+-giga,substance abuse-fast,substance abuse-giga,homelessness-fast,homelessness-giga,bullying-fast,bullying-giga
2183,The Soloist,,https://www.commonsensemedia.org/movie-reviews...,&quot;The Soloist&quot; manages to avoid the d...,PG-13,http://www.imdb.com/title/tt0821642/,[Movie],https://d2e111jq13me73.cloudfront.net/sites/de...,age 12+,5 reviews,...,0.903415,0.924851,0.943240,0.966900,0.929849,0.934331,0.947563,0.947846,0.923872,0.941183
9185,When Calls the Heart: The Heart of Homecoming,Nice people in a picturesque setting with solv...,https://www.commonsensemedia.org/movie-reviews...,Nice people in a picturesque setting with solv...,NR,http://www.imdb.com/title/tt7787044/,"[Movie, Magic and Fantasy, Brothers and Sister...",https://d2e111jq13me73.cloudfront.net/sites/de...,age 6+,1 review,...,0.944284,0.944606,0.954233,0.970782,0.956906,0.940089,0.966598,0.957716,0.948938,0.948336
3904,The Land Before Time: Journey of the Brave,There's a gentle sweetness to this story which...,https://www.commonsensemedia.org/movie-reviews...,There's a gentle sweetness to this story which...,G,http://www.imdb.com/title/tt4431254/?ref_=fn_a...,"[Movie, Dinosaurs, Adventures, Friendship]",https://d2e111jq13me73.cloudfront.net/sites/de...,age 6+,1 review,...,0.938190,0.932267,0.967248,0.972862,0.958870,0.940502,0.968789,0.959317,0.949451,0.942764
6196,Ice Age,This movie is a solid choice for kids old enou...,https://www.commonsensemedia.org/movie-reviews...,This movie is a solid choice for kids old enou...,PG,http://www.imdb.com/title/tt0268380,"[Movie, Adventures, Friendship, Misfits and Un...",https://d2e111jq13me73.cloudfront.net/sites/de...,age 5+,30 reviews,...,0.939788,0.933035,0.965555,0.973085,0.959116,0.937112,0.966989,0.952873,0.950248,0.940077


In [49]:
print("--- bullying")
print(df_csm.sort_values('bullying-fast', ascending=False)[["title", "bullying-fast", 'summary']].head(15))
print(df_csm.sort_values('bullying-giga', ascending=False)[["title", "bullying-giga", 'summary']].head(15))

print("--- sustainability")
print(df_csm.sort_values('sustainability-fast', ascending=False)[["title", "sustainability-fast", 'summary']].head(15))
print(df_csm.sort_values('sustainability-giga', ascending=False)[["title", "sustainability-giga", 'summary']].head(15))

print("--- civic engagement")
print(df_csm.sort_values('civic engagement-giga', ascending=False)[["title", "civic engagement-giga", 'summary']].head(15))
print(df_csm.sort_values('civic engagement-fast', ascending=False)[["title", "civic engagement-fast", 'summary']].head(15))

print("--- LGBTQ+")
print(df_csm.sort_values('LGBTQ+-fast', ascending=False)[["title", "LGBTQ+-fast", 'summary']].head(15))
print(df_csm.sort_values('LGBTQ+-giga', ascending=False)[["title", "LGBTQ+-giga", 'summary']].head(15))

print("--- homelessness")
print(df_csm.sort_values('homelessness-fast', ascending=False)[["title", "homelessness-fast", 'summary']].head(15))
print(df_csm.sort_values('homelessness-giga', ascending=False)[["title", "homelessness-giga", 'summary']].head(15))

print("--- substance abuse")
print(df_csm.sort_values('substance abuse-fast', ascending=False)[["title", "substance abuse-fast", 'summary']].head(15))
print(df_csm.sort_values('substance abuse-giga', ascending=False)[["title", "substance abuse-giga", 'summary']].head(15))



--- bullying
                                             title  bullying-fast  \
6626                                Paths of Glory       0.968626   
6930                   And the Children Shall Lead       0.967896   
4266                                 Psychokinesis       0.967787   
6164                                     One of Us       0.967493   
8270                              Dancing in Jaffa       0.967468   
631                                       Primeval       0.967186   
5733                                        Animas       0.967015   
4973  Digital Nation: Life on the Virtual Frontier       0.966741   
8977                               Curse of Chucky       0.966522   
9299                                         Shine       0.966473   
5215                       The Initiation of Sarah       0.966426   
148                                         Norman       0.966406   
9115                     Batman: Assault on Arkham       0.966154   
8378                 

                                               title  substance abuse-giga  \
4481                                 After Innocence              0.966853   
4463                                Grass Is Greener              0.966350   
1432                                            13th              0.966048   
5536                 Mankind: The Story of All of Us              0.965940   
8270                                Dancing in Jaffa              0.965423   
2701                                  Audrie & Daisy              0.965072   
7089                                      Code Black              0.964681   
6349                      An Officer and a Gentleman              0.964579   
8026  America the Beautiful 2: The Thin Commandments              0.963459   
4973    Digital Nation: Life on the Virtual Frontier              0.963383   
3398                                The Dream Is Now              0.963293   
1028                         How to Survive a Plague            